In [124]:
import numpy as np
import torch
from torch.nn import Sequential, Conv1d, ReLU, MaxPool2d, Linear, Sigmoid, Flatten

In [125]:
net = Sequential(
    Conv1d(1, 16, kernel_size=(10,), stride=(1,)),
    ReLU(),
    MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    Flatten(),
    Linear(in_features=65496, out_features=1, bias=True),
    Sigmoid()
)

In [130]:
s = 2**14
path = "data/train/benign/004fb7bd7b6da390b0a3d2acc6790cfa8da9c28fa5235bbf8659e1e0c4944e3d"

with open(path, 'rb') as f:
    data = f.read()

X = np.zeros(s, dtype=np.float32) # the input should be a float tensor even if the raw input is bytes 
l = len(data)

if l < s:
    X[:l] = np.frombuffer(data, dtype=np.uint8)
elif l > s:
    # prerare for mean-pooling
    window_size = int(np.ceil(l / s))
    byte_group_count = int(np.ceil(l / window_size))
    padded_data_length = byte_group_count * window_size
    data_array = np.zeros(padded_data_length)
    data_array[:l] = np.frombuffer(data, dtype=np.uint8)
    # mean pooling
    data_array = data_array.reshape((-1, window_size))
    X[:byte_group_count] = np.mean(data_array, axis=1)
else: # l == s
    np.frombuffer(data, dtype=np.uint8)    
    
# scale to 0..1 float and convert to batch tensor
X /= 255.
X = torch.from_numpy(X).unsqueeze(0).unsqueeze(0)

In [131]:
X, X.shape

(tensor([[[0.2690, 0.0016, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]]),
 torch.Size([1, 1, 16384]))

In [132]:
net(X)

tensor([[0.5248]], grad_fn=<SigmoidBackward0>)